In [21]:
"""
https://wg-cy.tistory.com/54


"""

import pandas as pd
import numpy as np
import requests
import websocket
import time
import json
from bs4 import BeautifulSoup
from lxml import etree
import re
import time
from tqdm import tqdm
import os
from urllib.request import urlopen
from io import BytesIO
from datetime import datetime
from dateutil.relativedelta import relativedelta
from html_table_parser import parser_functions as parser
import re

pd.set_option('display.max_columns', 10)                       # 출력할 열의 최대개수
pd.set_option('display.max_colwidth', 20)                      # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width', True)

# 최근 영업일

In [105]:
url = 'https://finance.naver.com/sise/sise_deposit.nhn'
response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
dom = etree.HTML(str(soup))
xpath = '//*[@id="type_0"]/div/ul[2]/li/span'
biz_day = dom.xpath(xpath)[0].text
biz_day = re.sub(r'[^0-9]', '', biz_day)
print(biz_day)

20220622


# 1. 업종분류 현황

In [122]:
sector = pd.DataFrame()

for mkt in ['STK', 'KSQ']:
    # 업종분류 현황 OTP 생성
    gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_otp_data = {
        'mktId' : mkt,
        'trdDd' : biz_day,
        'money' : '1',
        'csvxls_isNo' : 'false',
        'name' : 'fileDown',
        'url' : 'dbms/MDC/STAT/standard/MDCSTAT03901'
    }
    headers={'referer':'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

    response1 = requests.post(gen_otp_url, gen_otp_data, headers=headers)
    otp = response1.text

    # 업종분류 현황 다운로드
    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    down_sector = requests.post(down_url, {'code' : otp}, headers=headers)
    down_sector = pd.read_csv(BytesIO(down_sector.content), encoding='EUC-KR')
    print(len(down_sector))
    sector = pd.concat([sector, down_sector])

print(len(sector))
sector.head()

941
1559
2500


,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,6510,-230,-3.41,304813140450
1,006840,AK홀딩스,KOSPI,기타금융,16100,-500,-3.01,213285732100
2,027410,BGF,KOSPI,기타금융,4080,-175,-4.11,390524507280
3,282330,BGF리테일,KOSPI,유통업,174500,-2500,-1.41,3016041597000
4,138930,BNK금융지주,KOSPI,기타금융,6820,-250,-3.54,2222878377720


In [120]:
sector.to_csv('data/krx_sector.csv', index=False, encoding='cp949')

# 개별종목 지표

In [131]:
# 개별종목 지표 OTP 발급
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'seachType' : 1,
    'mktId' : 'ALL',
    'trdDd' : biz_day,
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT03501'
}

headers={'referer':'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}

response1 = requests.post(gen_otp_url, gen_otp_data, headers=headers)
otp = response1.text

# 개별종목 지표 데이터 다운로드
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_ind = requests.post(down_url, {'code' : otp}, headers=headers)
down_ind = pd.read_csv(BytesIO(down_ind.content), encoding='EUC-KR')
down_ind['종목명'] = down_ind['종목명'].apply(lambda x: x[:-1])
print(len(down_ind))
down_ind.head()

2452


,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,060310,3S,2940,15,0.51,16.0,183.75,NaN,NaN,792.0,3.71,0,0.00
1,095570,AJ네트웍스,6510,-230,-3.41,1707.0,3.81,973.0,6.69,8075.0,0.81,270,4.15
2,006840,AK홀딩스,16100,-500,-3.01,NaN,NaN,NaN,NaN,45961.0,0.35,200,1.24
3,054620,APS홀딩스,9150,-640,-6.54,1179.0,7.76,NaN,NaN,10088.0,0.91,100,1.09
4,265520,AP시스템,16600,-1400,-7.78,3932.0,4.22,3622.0,4.58,12713.0,1.31,240,1.45


In [132]:
down_ind.to_csv('data/krx_ind.csv', index=False, encoding='cp949')

In [109]:
print(len(sector), len(down_ind))

2500 2452


# 섹터 정보

In [79]:
url = 'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20190607&sec_cd=G10'
response = urlopen(url)
data = json.load(response)
sector_code = pd.DataFrame(data['sector'])['SEC_CD'].unique().tolist()

data_sector = pd.DataFrame()

for c in tqdm(sector_code, total = len(sector_code), ncols=100):
    url = 'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={}&sec_cd={}'.format(biz_day, c)
    response = urlopen(url)
    data = json.load(response)
    data = pd.DataFrame(data['list'])

    data_sector = pd.concat([data_sector, data])
    
    time.sleep(1)

data_sector.head()

dict_keys(['info', 'list', 'sector', 'size'])

In [81]:
if not os.path.exists('data'):
    os.mkdir('data')

data_sector.to_csv('data/KOR_sector.csv', index=False, encoding='cp949')

['G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45']

# 데이터 정리

In [161]:
# 컬럼 교집합
inter_cols = set(sector.columns) & set(down_ind.columns)
inter_cols

{'대비', '등락률', '종가', '종목명', '종목코드'}

In [162]:
# 한쪽 데이터에만 있는 종목
# -> 해당 종목들은 선박펀드, 광물펀드, 해외종목 등 일반적이지 않은 종목들이므로 제외하는 것이 좋습니다.
one_side = set(sector['종목명']) ^ set(down_ind['종목명'])
print(len(one_side))
print(one_side)

56
{'NH프라임리츠', '글로벌에스엠', '로스웰', 'SBI핀테크솔루션즈', '한국패러랠', 'NH올원리츠', '디앤디플랫폼리츠', '마스턴프리미어리츠', '세기상사 [락', '제이알글로벌리츠', '베트남개발1', '바다로19호', '미투젠', '이지스밸류리츠', '코스모화학', '씨케이에이치', '맵스리얼티1', '이스트아시아홀딩스', '프레스티지바이오파마', '네오이뮨텍', '신한알파리츠', '골든센츄리', '세기상사', '크리스탈신소재', '코람코에너지리츠', '인카금융서비스 [락', '애머릿지', '에이리츠', '잉글우드랩', '현대사료', '이지스레지던스리츠', '엑세스바이오', '이리츠코크렙', 'GRT', '한국ANKOR유전', '엘브이엠씨홀딩스', '코오롱티슈진', '코람코더원리츠', '인카금융서비스', '케이탑리츠', '현대사료 (액', '모두투어리츠', '헝셩그룹', '윙입푸드', '맥쿼리인프라', '신한서부티엔디리츠', '코스모화학 [락', 'SK리츠', 'JTC', '소마젠', '미래에셋맵스리츠', '컬러레이', '오가닉티코스메틱', '롯데리츠', '미래에셋글로벌리츠', 'ESR켄달스퀘어리츠'}


In [172]:
KOR_ticker = pd.merge(sector, down_ind, how='inner', on=list(inter_cols))
KOR_ticker = KOR_ticker.sort_values('시가총액', ascending=False)
KOR_ticker

,종목코드,종목명,시장구분,업종명,종가,...,선행 PER,BPS,PBR,주당배당금,배당수익률
415,005930,삼성전자,KOSPI,전기전자,57600,...,8.14,43611.0,1.32,1444,2.51
84,373220,LG에너지솔루션,KOSPI,전기전자,401000,...,79.06,39831.0,10.07,0,0.00
147,000660,SK하이닉스,KOSPI,전기전자,92200,...,5.30,90394.0,1.02,1540,1.67
409,207940,삼성바이오로직스,KOSPI,의약품,817000,...,100.17,74551.0,10.96,0,0.00
416,005935,삼성전자우,KOSPI,전기전자,53000,...,NaN,NaN,NaN,1445,2.73
...,...,...,...,...,...,...,...,...,...,...,...
1719,176440,에이치엔티,KOSDAQ,IT부품,46,...,NaN,172.0,0.27,0,0.00
306,001527,동양2우B,KOSPI,비금속광물,12250,...,NaN,NaN,NaN,150,1.22
1495,032685,소프트센우,KOSDAQ,유통,25850,...,NaN,NaN,NaN,0,0.00
307,001529,동양3우B,KOSPI,비금속광물,24250,...,NaN,NaN,NaN,150,0.62


In [173]:
KOR_ticker = KOR_ticker[KOR_ticker['종목명'].apply(lambda x: '스팩' not in x)] # 스팩주 제외
KOR_ticker = KOR_ticker[KOR_ticker['종목코드'].apply(lambda x: x[-1] == '0')] # 우선주 제외
KOR_ticker.reset_index(inplace=True, drop=True)
KOR_ticker

,종목코드,종목명,시장구분,업종명,종가,...,선행 PER,BPS,PBR,주당배당금,배당수익률
0,005930,삼성전자,KOSPI,전기전자,57600,...,8.14,43611.0,1.32,1444,2.51
1,373220,LG에너지솔루션,KOSPI,전기전자,401000,...,79.06,39831.0,10.07,0,0.00
2,000660,SK하이닉스,KOSPI,전기전자,92200,...,5.30,90394.0,1.02,1540,1.67
3,207940,삼성바이오로직스,KOSPI,의약품,817000,...,100.17,74551.0,10.96,0,0.00
4,051910,LG화학,KOSPI,화학,553000,...,15.27,278455.0,1.99,12000,2.17
...,...,...,...,...,...,...,...,...,...,...,...
2263,121890,에스디시스템,KOSDAQ,정보기기,739,...,NaN,838.0,0.88,0,0.00
2264,058420,제이웨이,KOSDAQ,오락·문화,2160,...,NaN,NaN,NaN,0,0.00
2265,347140,케이프이에스제4호,KOSDAQ,금융,2090,...,NaN,NaN,NaN,0,0.00
2266,050540,한국코퍼레이션,KOSDAQ,기타서비스,822,...,NaN,2139.0,0.38,0,0.00


In [174]:
KOR_ticker.to_csv('data/KOR_ticker.csv', index=False, encoding='cp949')

# 수정주가

In [68]:
code_template = '{0:06d}'
KOR_ticker = pd.read_csv('data/KOR_ticker.csv', encoding='cp949')
code_template.format(KOR_ticker['종목코드'][0])

'005930'

In [179]:
if not os.path.exists('data/KOR_price'):
    os.makedirs('data/KOR_price')

In [186]:
i = 0
name = pd.pd.to_datetime([datetime.now().date()])
name

DatetimeIndex(['2022-06-25'], dtype='datetime64[ns]', freq=None)

In [201]:
str(end_time)

'2022-06-25'

In [70]:
code_template = '{0:06d}'
price_template = 'https://fchart.stock.naver.com/siseJson.nhn?symbol={}&requestType=1&startTime={}&endTime={}&timeframe=day'

end_time = datetime.now().date()
start_time = end_time - relativedelta(years=20)
end_time = re.sub(r'[^0-9]', '', str(end_time))
start_time = re.sub(r'[^0-9]', '', str(start_time))

for i in tqdm(range(len(KOR_ticker)), total=len(KOR_ticker), ncols=50):
    code = code_template.format(KOR_ticker['종목코드'][i])
    url = price_template.format(code, start_time, end_time)

    response = requests.get(url)
    data = json.loads(response.text.replace('\'', '\"'))
    data = pd.DataFrame(data[1:], columns=data[0])
    data = data.loc[:, ['날짜', '종가']]
    data.to_csv('data/KOR_price/{}_price.csv'.format(code), index=False, encoding='cp949')
    time.sleep(0.5)

  2%|▏          | 42/2268 [00:27<24:17,  1.53it/s]


JSONDecodeError: Expecting value: line 391 column 50 (char 12094)

In [71]:
data

,날짜,종가
0,20211103,193000
1,20211104,169000
2,20211105,170000
3,20211108,153500
4,20211109,147000
...,...,...
155,20220621,70600
156,20220622,68000
157,20220623,65200
158,20220624,66900


# 재무재표

In [28]:


table_class_info = {
    '포괄손익계산서' : 'us_table_ty1 h_fix zigbg_no',
    '재무상태표' : 'us_table_ty1 h_fix zigbg_no'
}
ticker = '005930'
fs_url_template = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}'

req = urlopen(fs_url_template.format(ticker))
html = req.read()

soup = BeautifulSoup(html, 'lxml')

# 원하는 table 값이 있는 class 넣기
soup_table = soup.find_all('table', attrs={'class': 'us_table_ty1 h_fix zigbg_no'})

table = parser.make2d(soup_table[3])
df = pd.DataFrame(table[1:], columns=table[0])
df

,IFRS(연결),2021/06,2021/09,2021/12,2022/03
0,IFRS(연결),2021/06,2021/09,2021/12,2022/03
1,자산,"3,847,777","4,104,207","4,266,212","4,393,270"
2,유동자산계산에 참여한 계정 펼...,"1,911,185","2,127,930","2,181,632","2,323,691"
3,재고자산,"335,924","378,017","413,844","475,907"
4,유동생물자산,,,,
...,...,...,...,...,...
62,자본잉여금,"44,039","44,039","44,039","44,039"
63,기타자본,543,685,868,931
64,기타포괄손익누계액,"-62,803","-18,657","-22,152","-7,458"
65,이익잉여금(결손금),"2,750,852","2,848,270","2,930,648","3,017,432"


In [37]:
code_template = '{0:06d}'
fs_url_template = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{}'


KOR_ticker = pd.read_csv('data/KOR_ticker.csv', encoding='cp949')

if not os.path.exists('data/KOR_fs'):
    os.makedirs('data/KOR_fs')
if not os.path.exists('data/KOR_fs_q'):
    os.makedirs('data/KOR_fs_q')

for i in tqdm(
    range(len(KOR_ticker)), total=len(KOR_ticker), ncols=50):
    code = code_template.format(KOR_ticker['종목코드'][i])

    req = urlopen(fs_url_template.format(code))
    html = req.read()

    soup = BeautifulSoup(html, 'lxml')

    # 원하는 table 값이 있는 class 넣기
    soup_table = soup.find_all('table', attrs={'class': 'us_table_ty1 h_fix zigbg_no'})

    tables = [parser.make2d(st) for st in soup_table]
    data = [pd.DataFrame(table[1:], columns=table[0]) for table in tables]
    time.sleep(0.2)

    # remove 전년동기, 전년동기(%)  
    for j in range(2):
        data[j] = data[j].iloc[:, :len(data[j].columns)-2]
        
    data_fs = pd.concat([data[j] for j in range(0, 6, 2)]) # yearly
    data_fs_q = pd.concat([data[j] for j in range(1, 6, 2)]) # quarterly

    
    # IFRS(연결) -> 계정
    data_fs = data_fs.rename(
        columns = {data_fs.columns[0]: '계정'})
    data_fs_q = data_fs_q.rename(
        columns = {data_fs_q.columns[0]: '계정'})

    # 12월 데이터만
    data_fs = data_fs.loc[:, [col_name for col_name in data_fs.columns\
        if col_name[-2:] == '12' or col_name == '계정']]

    # remove '계산에 참여한 계정 펼치기' '\xa0'
    for txt in ['계산에 참여한 계정 펼치기', '\xa0']:
        data_fs = data_fs.applymap(
            lambda x: x.replace(txt, '') if isinstance(x, str) else x
        )
        data_fs_q = data_fs_q.applymap(
            lambda x: x.replace(txt, '') if isinstance(x, str) else x
        )

    # remove 중복되는 계정명
    data_fs.drop_duplicates(
        subset=['계정'], keep=False, inplace=True)
    data_fs_q.drop_duplicates(
        subset=['계정'], keep=False, inplace=True)

    data_fs.reset_index(drop=True, inplace=True)
    data_fs_q.reset_index(drop=True, inplace=True)

    # str -> number 
    data_fs.iloc[:, 1:] = data_fs.iloc[:, 1:].applymap(lambda x: float(re.sub(r'[^0-9]', '', x)) if isinstance(x, str) else x)
    data_fs_q.iloc[:, 1:] = data_fs_q.iloc[:, 1:].applymap(lambda x: float(re.sub(r'[^0-9]', '', x)) if isinstance(x, str) else x)
    # for i in range(len(data)):
    #     data[i] = data[i].applymap(lambda x: float(re.sub(r'[^0-9]', '', x)) if isinstance(x, str) else x)
    
    # save
    data_fs.to_csv(
        'data/KOR_fs/{}_fs.csv'.format(code),
        index=False, encoding='cp949')
    data_fs_q.to_csv(
        'data/KOR_fs_q/{}_fs_q.csv'.format(code),
        index=False, encoding='cp949')

SyntaxError: invalid syntax (<ipython-input-37-8f3840c59950>, line 65)

# 가치지표

In [6]:
if not os.path.exists('data/KOR_value'): os.makedirs('data/KOR_value')

code_template = '{0:06d}'
KOR_ticker = pd.read_csv('data/KOR_ticker.csv', encoding='cp949')
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].apply(lambda x: code_template.format(x))

price_url_template = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}'
value_type = [
    '지배주주순이익',
    '자본',
    '영업활동으로인한현금흐름',
    '매출액'
]

not_generated = []
for i in tqdm(range(len(KOR_ticker)), total=len(KOR_ticker), ncols=50):
    # if i<=542:
    #     continue

    try:
        code = KOR_ticker['종목코드'][i]
        data_fs = pd.read_csv(
            'data/KOR_fs/{}_fs.csv'.format(code), encoding='cp949')

        value_index = []
        fs_account = data_fs['계정'].tolist()
        for vt in value_type:
            if vt in fs_account:
                value_index.append(data_fs.iloc[fs_account.index(vt), -1])
            else:
                value_index.append(-1)
        value_index = pd.Series(value_index)
        # value_index = data_fs.iloc[
        #     [data_fs['계정'].tolist().index(vt) for vt in value_type], -1]
        # value_index = value_index.set_index('계정')

        response = requests.get(price_url_template.format(code))

        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
        dom = etree.HTML(str(soup))

        # current price
        xpath = '//*[@id="svdMainChartTxt11"]'
        price = dom.xpath(xpath)[0].text
        price = re.sub(r'[^0-9]', '', price)
        price = int(price)
        price

        # share
        xpath = '//*[@id="svdMainGrid1"]/table/tbody/tr[7]/td[1]'
        share = dom.xpath(xpath)[0].text
        share = share.split('/')[0]
        share = re.sub(r'[^0-9]', '', share)
        share = int(share)
        share

        # value_index
        value_index = value_index.replace(0, np.nan)
        data_value = value_index.apply(lambda x: price / (x * 100000000 / share))
        data_value.index = ('PER', 'PBR', 'PCR', 'PSR')
        data_value = data_value.to_frame().transpose().reset_index(drop=True)
        data_value[data_value < 0] = None

        # save
        data_value.to_csv('data/KOR_value/{}_value.csv'.format(code), index=False, encoding='cp949')
    except TypeError: # : unsupported operand type(s) for /: 'str' and 'int'
        not_generated.append(code)
print('가치지표 생성되지 않은 종목:\n', not_generated)

100%|█████████| 2268/2268 [16:31<00:00,  2.29it/s]

가치지표 생성되지 않은 종목:
 176440


In [7]:
print('가치지표 생성되지 않은 종목:\n', not_generated)

가치지표 생성되지 않은 종목:
 ['099750', '289080', '054220', '227950', '060310', '004310', '003610', '030960', '097870', '092440', '021880', '042520', '001080', '002630', '021820', '033200', '169330', '131100', '008870', '008110', '050860', '189690', '190650', '019570', '154030', '222160', '093240', '067010', '024850', '020180', '018500', '093380', '033250', '017250']


# Dart

In [99]:
import dart_fss as dart
api_key = 'a57617cb94cef323f12dab58dc9ea557fb28196d'
dart.set_api_key(api_key=api_key)

'a57617cb94cef323f12dab58dc9ea557fb28196d'

In [100]:
corp_list = dart.get_corp_list()
corp_list

Output()

In [2]:
# 삼성전자 기업코드로 2021년 1월 1일부터 현재날짜 사이에 나온 사업보고서 가져오기
corp_code_samsung = '00126380'
fs = dart.fs.extract(corp_code=corp_code_samsung, bgn_de='20220101', fs_tp=('bs', 'cis', 'cf'), report_tp='annual', lang='ko')

Annual reports:   0%|          | 0/1 [00:00<?, ?report/s]

KeyboardInterrupt: 

In [1]:
fs

NameError: name 'fs' is not defined